In [11]:
import sys
print(sys.executable)

/opt/homebrew/opt/python@3.12/bin/python3.12


In [12]:
!{sys.executable} -m pip install -q -U watermark


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [13]:
!{sys.executable} -m pip install -q -U torch


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [14]:
import random
random.seed(10)

In [15]:
# Imports
import re
import math
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from random import *

## Loading the Text Data

In [16]:
project_text = open("texto.txt", "r").read()

## Preprocessing the Text Data and Building the Vocabulary

In [17]:
# Filtering special characters: '.', ',', '?', '!'
sentences = re.sub("[.,!?\\-]", "", project_text.lower()).split("\n") 

# Split the phrases into words and create a list of words
word_list = list(set(" ".join(sentences).split()))

# Inicialize the word dictionary with BERT's special tokens
word_dict = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}

# Add the words to the dictionary and create indexes
for i, w in enumerate(word_list):
    word_dict[w] = i + 4
    
# Invert the order and put the indexes as keys and the words as values in the dictionary
number_dict = {i: w for i, w in enumerate(word_dict)}

vocab_size = len(word_dict)
token_list = list()

In [18]:
# Loop for each sentence to create the token list
for sentence in sentences:
    arr = [word_dict[s] for s in sentence.split()]
    token_list.append(arr)

## Hyperparameters Definition

In [19]:
batch_size = 6
n_segments = 2
dropout = 0.2

maxlen = 100 
max_pred = 7
n_layers = 6 
n_heads = 12
d_model = 768
d_ff = d_model * 4
d_k = d_v = 64 

NUM_EPOCHS = 50

## Batch Data Creation and Special Tokens Application

In [20]:
# Define the function to create data batches
def dsa_make_batch():
    
    # Initialize the batch as an empty list
    batch = []
    
    # Initialize counters for positive and negative examples
    positive = negative = 0
    
    # Continue until half the batch consists of positive examples and the other half of negative examples
    while positive != batch_size/2 or negative != batch_size/2:
        
        # Randomly choose indices for two sentences
        tokens_a_index, tokens_b_index = randrange(len(sentences)), randrange(len(sentences))
        
        # Retrieve the corresponding tokens for these indices
        tokens_a, tokens_b = token_list[tokens_a_index], token_list[tokens_b_index]
        
        # Prepare input ids by adding special tokens [CLS] and [SEP]
        input_ids = [word_dict['[CLS]']] + tokens_a + [word_dict['[SEP]']] + tokens_b + [word_dict['[SEP]']]
        
        # Define segment ids to differentiate the two sentences
        segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (len(tokens_b) + 1)
        
        # Calculate the number of predictions to make (15% of tokens)
        n_pred =  min(max_pred, max(1, int(round(len(input_ids) * 0.15)))) 
        
        # Identify candidate positions for masking that are not [CLS] or [SEP]
        cand_maked_pos = [i for i, token in enumerate(input_ids) if token != word_dict['[CLS]'] and token != word_dict['[SEP]']]
        
        # Shuffle candidate positions
        shuffle(cand_maked_pos)
        
        # Initialize lists for masked tokens and their positions
        masked_tokens, masked_pos = [], []
        
        # Mask tokens until reaching the desired number of predictions
        for pos in cand_maked_pos[:n_pred]:
            masked_pos.append(pos)
            masked_tokens.append(input_ids[pos])
            
            # Random mask
            if random() < 0.8:  
                input_ids[pos] = word_dict['[MASK]'] 
            
            # Replace with another token 10% of the time (20% of the remaining cases)
            elif random() < 0.5:  
                index = randint(0, vocab_size - 1) 
                input_ids[pos] = word_dict[number_dict[index]] 
        
        # Add zero padding to input ids and segment ids to reach maximum length
        n_pad = maxlen - len(input_ids)
        input_ids.extend([0] * n_pad)
        segment_ids.extend([0] * n_pad)
        
        # Add zero padding to masked tokens and their positions if necessary
        if max_pred > n_pred:
            n_pad = max_pred - n_pred
            masked_tokens.extend([0] * n_pad)
            masked_pos.extend([0] * n_pad)
        
        # Add to the batch as a positive example if sentences are consecutive
        if tokens_a_index + 1 == tokens_b_index and positive < batch_size / 2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, True]) 
            positive += 1
        
        # Add to the batch as a negative example if sentences are not consecutive
        elif tokens_a_index + 1 != tokens_b_index and negative < batch_size / 2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, False]) 
            negative += 1
    
    # Return the complete batch
    return batch

In [21]:
def dsa_get_attn_pad_masked(seq_q, seq_k):    
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    pad_attn_masked = seq_k.data.eq(0).unsqueeze(1)
    
    return pad_attn_masked.expand(batch_size, len_q, len_k)


In [22]:

batch = dsa_make_batch()

input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))
dsa_get_attn_pad_masked(input_ids, input_ids)[0][0], input_ids[0]

(tensor([False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True]),
 tensor([ 1, 21, 45, 41, 58, 28,  3, 25, 37, 34, 67, 55,  2,  3, 57, 17, 13, 33,
         24, 19,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  

## Model building

In [23]:
def gelu(x):
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))

### 1- Embedding

In [24]:
# Embedding Class
class Embedding(nn.Module):
    
    # Constructor method
    def __init__(self):
        super(Embedding, self).__init__()
        
        # Token embedding
        self.tok_embed = nn.Embedding(vocab_size, d_model)  
        
        # Position embedding
        self.pos_embed = nn.Embedding(maxlen, d_model)  
        
        # Segment (token type) embedding
        self.seg_embed = nn.Embedding(n_segments, d_model)  
        
        # Layer normalization
        self.norm = nn.LayerNorm(d_model)

    # Forward method
    def forward(self, x, seg):
        seq_len = x.size(1)
        pos = torch.arange(seq_len, dtype=torch.long)
        
        # (seq_len,) -> (batch_size, seq_len)
        pos = pos.unsqueeze(0).expand_as(x)  
        embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
        
        return self.norm(embedding)


### 2- Scaled Dot Product Attention

In [25]:
# Define the class for performing scaled dot-product attention
class ScaledDotProductAttention(nn.Module):
    
    # Class initialization method
    def __init__(self):
        # Initialize the base class
        super(ScaledDotProductAttention, self).__init__()

    # Forward method to define the forward pass of data
    def forward(self, Q, K, V, attn_mask):
        # Calculate attention scores as the product of Q and K, normalized by the key size
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k)
        
        # Apply attention mask to prevent attention to certain tokens
        scores.masked_fill_(attn_mask, -1e9)
        
        # Apply softmax to obtain normalized attention weights
        attn = nn.Softmax(dim=-1)(scores)
        
        # Multiply attention weights by V to obtain context
        context = torch.matmul(attn, V)
        
        # Return the context and attention weights
        return context, attn


### 3- Multi-Head Attention

In [26]:
# Define the class for performing multi-head attention
class MultiHeadAttention(nn.Module):
    
    def __init__(self) -> None:
        
        # Initialize the base class
        super(MultiHeadAttention, self).__init__()
        
        # Define the weight matrix for queries Q
        self.W_Q = nn.Linear(d_model, d_k * n_heads)
        
        # Define the weight matrix for keys K
        self.W_K = nn.Linear(d_model, d_k * n_heads)
        
        # Define the weight matrix for values V
        self.W_V = nn.Linear(d_model, d_v * n_heads)

    # Forward method to define the forward pass of data
    def forward(self, Q, K, V, attn_mask):
        
        # Save input Q for residual connection and obtain the batch size
        residual, batch_size = Q, Q.size(0)
        
        # Process Q through W_Q and reshape to have [n_heads] in the second dimension
        q_s = self.W_Q(Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)
        
        # Process K through W_K and reshape to have [n_heads] in the second dimension
        k_s = self.W_K(K).view(batch_size, -1, n_heads, d_k).transpose(1,2)
        
        # Process V through W_V and reshape to have [n_heads] in the second dimension
        v_s = self.W_V(V).view(batch_size, -1, n_heads, d_v).transpose(1,2)
        
        # Adapt attn_mask to be compatible with the dimensions of q_s, k_s, v_s
        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1)
        
        # Compute scaled dot-product attention and context for each attention head
        context, attn = ScaledDotProductAttention()(q_s, k_s, v_s, attn_mask)
        
        # Reorganize the context to combine attention heads and revert to the original format
        context = context.transpose(1,2).contiguous().view(batch_size, -1, n_heads * d_v)
        
        # Apply a linear transformation to the combined context
        output = nn.Linear(n_heads * d_v, d_model)(context)
        
        # Normalize the output layer and add the residual connection
        return nn.LayerNorm(d_model)(output + residual), attn

In [27]:
# Create the Embedding object
emb = Embedding()

# Generate the embeddings
embeds = emb(input_ids, segment_ids)

# Generate the attention mask
attenM = dsa_get_attn_pad_masked(input_ids, input_ids)

# Generate MultiHeadAttention
MHA = MultiHeadAttention()(embeds, embeds, embeds, attenM)

output, A = MHA

### 4- Feedforward Posicional

In [28]:
# Define the class for the Position-wise Feed Forward network
class PoswiseFeedForward(nn.Module):
    
    def __init__(self) -> None:
        
        # Initialize the base class
        super(PoswiseFeedForward, self).__init__()
        
        # First linear layer increasing dimension from d_model to d_ff
        self.fc1 = nn.Linear(d_model, d_ff)
        
        # Second linear layer reducing dimension from d_ff back to d_model
        self.fc2 = nn.Linear(d_ff, d_model)

    # Forward method to define the forward pass of data
    def forward(self, x):
        
        # Apply the first linear transformation, followed by the GELU activation function,
        # and then the second linear transformation
        return self.fc2(gelu(self.fc1(x)))


### 5- Encoder Layer

In [29]:
# Define the class for the encoder layer
class EncoderLayer(nn.Module):
    
    def __init__(self) -> None:
        
        # Initializes the base class
        super(EncoderLayer, self).__init__()
        
        # Instantiates multi-head attention for encoder self-attention
        self.enc_self_attn = MultiHeadAttention()
        
        # Instantiates the Positional Feed Forward Network to be used after self-attention
        self.pos_ffn = PoswiseFeedForward()

    # Forward method to define the forward pass of the data
    def forward(self, enc_inputs, enc_self_attn_mask):
        
        # Applies self-attention to the input data
        enc_inputs, atnn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask)
        
        # After self-attention, passes the result through the Positional Feed Forward Network
        enc_inputs = self.pos_ffn(enc_inputs)
        
        # Returns the encoder output and attention weights
        return enc_inputs, atnn


### 6 - Final LLM Architecture (BERT model)

In [30]:
class BERT(nn.Module):
    
    def __init__(self) -> None:
        super(BERT, self).__init__()
        self.embedding = Embedding()
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])
        self.fc = nn.Linear(d_model, d_model)
        self.activ1 = nn.Tanh()
        self.linear = nn.Linear(d_model, d_model)
        self.activ2 = gelu
        self.norm = nn.LayerNorm(d_model)
        self.classifier = nn.Linear(d_model, 2)
        
        embed_weight = self.embedding.tok_embed.weight
        n_vocab, n_dim = embed_weight.size()
        
        self.decoder = nn.Linear(n_dim, n_vocab, bias=False)
        self.decoder.weight = embed_weight
        self.decoder_bias = nn.Parameter(torch.zeros(n_vocab))

    def forward(self, input_ids, segment_ids, masked_pos):
        output = self.embedding(input_ids, segment_ids)
        enc_self_attn_mask = dsa_get_attn_pad_masked(input_ids, input_ids)
        for layer in self.layers:
            output, enc_self_attn = layer(output, enc_self_attn_mask)
        
        h_pooled = self.activ1(self.fc(output[:, 0]))
        logits_clsf = self.classifier(h_pooled)
        masked_pos = masked_pos[:, :, None].expand(-1, -1, output.size(-1))
        h_masked = torch.gather(output, 1, masked_pos)
        h_masked = self.norm(self.activ2(self.linear(h_masked)))
        logits_lm = self.decoder(h_masked) + self.decoder_bias
        
        return logits_lm, logits_clsf

## LLM training and evaluation

In [31]:
my_model = BERT()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(my_model.parameters(), lr = 0.001)
batch = dsa_make_batch()

input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))

In [32]:
%%time

# Starts the training loop for a defined number of epochs
for epoch in range(NUM_EPOCHS):
    # Resets the optimizer gradients to avoid accumulation from previous epochs
    optimizer.zero_grad()
    
    # Passes the input data through the model and obtains logits for 
    # masked language modeling and next sentence classification
    logits_lm, logits_clsf = my_model(input_ids, segment_ids, masked_pos)
    
    # Computes the loss for the masked language modeling task by comparing 
    # the predicted logits with the actual tokens
    loss_lm = criterion(logits_lm.transpose(1,2), masked_tokens)
    
    # Computes the average loss for normalization
    loss_lm = (loss_lm.float()).mean()
    
    # Computes the loss for the next sentence classification task
    loss_clsf = criterion(logits_clsf, isNext)
    
    # Sums the losses from both tasks to obtain the total loss
    loss = loss_lm + loss_clsf
    
    # Displays the current epoch and total loss
    print(f'Epoch: {epoch + 1} | Loss {loss:.4f}')
    
    # Performs backpropagation to compute gradients
    loss.backward()
    
    # Updates the model parameters based on the computed gradients
    optimizer.step()


Epoch: 1 | Loss 90.3972
Epoch: 2 | Loss 86.4410
Epoch: 3 | Loss 321.5726
Epoch: 4 | Loss 27.3724
Epoch: 5 | Loss 76.5048
Epoch: 6 | Loss 63.3785
Epoch: 7 | Loss 40.7729
Epoch: 8 | Loss 43.8030
Epoch: 9 | Loss 19.2123
Epoch: 10 | Loss 18.8235
Epoch: 11 | Loss 20.8981
Epoch: 12 | Loss 17.8936
Epoch: 13 | Loss 17.8925
Epoch: 14 | Loss 17.5576
Epoch: 15 | Loss 18.1180
Epoch: 16 | Loss 18.5692
Epoch: 17 | Loss 19.7877
Epoch: 18 | Loss 19.7330
Epoch: 19 | Loss 19.8825
Epoch: 20 | Loss 19.9438
Epoch: 21 | Loss 17.1900
Epoch: 22 | Loss 19.2765
Epoch: 23 | Loss 18.9486
Epoch: 24 | Loss 17.8177
Epoch: 25 | Loss 19.1062
Epoch: 26 | Loss 17.8007
Epoch: 27 | Loss 18.4139
Epoch: 28 | Loss 18.7882
Epoch: 29 | Loss 16.5637
Epoch: 30 | Loss 18.2306
Epoch: 31 | Loss 17.5891
Epoch: 32 | Loss 18.1362
Epoch: 33 | Loss 19.4828
Epoch: 34 | Loss 17.3118
Epoch: 35 | Loss 17.9853
Epoch: 36 | Loss 19.0028
Epoch: 37 | Loss 20.2405
Epoch: 38 | Loss 18.5852
Epoch: 39 | Loss 17.2323
Epoch: 40 | Loss 17.7292
Epoch: 4

## Extracting Predictions from the Trained LLM

In [33]:
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(batch[0]))
print(dsa_texto)
print([number_dict[w.item()] for w in input_ids[0] if number_dict[w.item()] != '[PAD]'])

'Olá, como vai? Eu sou a Camila.\n'
'Olá, Camila, meu nome é Fernando. Muito prazer.\n'
'Prazer em conhecer você também. Como você está hoje?\n'
'Ótimo. Meu time de futebol venceu a competição.\n'
'Uau, Parabéns Fernando!\n'
'Obrigado Camila.\n'
'Vamos comer uma pizza mais tarde para celebrar?\n'
'Claro. Você recomenda algum restaurante Camila?\n'
'Sim, abriu um restaurante novo e dizem que a pizza de banana é fenomenal.\n'
'Ok. Nos encontramos no restaurante às sete da noite, pode ser?\n'
'Pode sim. Nos vemos mais tarde então.'
['[CLS]', "'olá", '[MASK]', 'vai', 'eu', 'sou', 'a', "camila\\n'", '[SEP]', "'ok", 'nos', 'encontramos', 'no', 'restaurante', 'às', 'sete', '[MASK]', 'noite', 'pode', "'olá", '[SEP]']


In [36]:
logits_lm, logits_clsf = my_model(input_ids, segment_ids, masked_pos)
logits_lm = logits_lm.data.max(2)[1][0].data.numpy()
print('Actual Masked Tokens List: ', [pos.item() for pos in masked_tokens[0] if pos.item() != 0])
print('Predicted Masked Tokens List: ', [pos for pos in logits_lm if pos != 0])

Actual Masked Tokens List:  [55, 57, 37]
Predicted Masked Tokens List:  [np.int64(18), np.int64(18)]


In [37]:
# Extracts the next token predictions
logits_clsf = logits_clsf.data.max(1)[1].data.numpy()[0]
print('isNext (Actual Value): ', True if isNext else False)
print('isNext (Predicted Value): ', True if logits_clsf else False)


isNext (Actual Value):  False
isNext (Predicted Value):  False
